# Day 1 실습 2: 전처리 데이터 품질 점검

전처리된 `processed_data/` 폴더를 불러와 데이터 품질을 빠르게 검토합니다.

### 목차
1. 데이터 로드
2. 필수 컬럼 및 결측치 확인
3. 텍스트 길이/중복 점검
4. RAFT 템플릿 상태 확인
5. 메타데이터 요약
6. 다음 단계

## 1. 데이터 로드
- Colab에서 Google Drive에 저장한 경우 자동으로 마운트합니다.
- 로컬 환경이라면 기본 경로 `processed_data/`를 그대로 사용합니다.

In [ ]:
from datasets import load_dataset
from huggingface_hub import login
login(token="")  # HF Access Token 입력

train_df = load_dataset("ryanu/raft-processed", split="train")
print(train_df)


In [ ]:
# import json
# from datasets import load_from_disk

# DATA_DIR = "processed_data"
# try:
#     from google.colab import drive  # type: ignore
#     drive.mount('/content/drive')
#     DATA_DIR = "/content/drive/MyDrive/exaone_day1/processed_data"
#     print(f"✅ Google Drive 마운트: {DATA_DIR}")
# except ModuleNotFoundError:
#     print("📁 로컬 환경에서 실행 중입니다. DATA_DIR=processed_data")
# except NotImplementedError:
#     print("⚠️ 이 환경에서는 drive.mount()를 사용할 수 없습니다. 필요 시 DATA_DIR를 직접 수정하세요.")

# DATASET_PATH = DATA_DIR
# METADATA_PATH = f"{DATA_DIR}/metadata.json"

# hf_dataset = load_from_disk(DATASET_PATH)
# train_df = hf_dataset.to_pandas()

# with open(METADATA_PATH, "r", encoding="utf-8") as f:
#     metadata = json.load(f)

# print(f"✅ 데이터 로드 완료: {len(train_df):,}개 샘플")
# print(f"   - 컬럼: {train_df.columns.tolist()}")


## 2. 필수 컬럼 및 결측치 확인
Instruction/Input/Output 컬럼이 모두 존재하는지 확인합니다.

In [ ]:
# Hugging Face Dataset → pandas DataFrame 변환
train_df = train_df.to_pandas()

required_columns = {"text", "question", "answer"}
missing_columns = required_columns - set(train_df.columns)
if missing_columns:
    raise ValueError(f"필수 컬럼이 없습니다: {missing_columns}")

print("✅ 필수 컬럼 존재 확인")
print("결측치 개수")
print(train_df[sorted(required_columns)].isnull().sum())

## 3. 텍스트 길이 및 중복 점검
각 컬럼의 길이 범위와 중복 여부를 간단히 살펴봅니다.

In [ ]:
length_stats = {
    "text": train_df["text"].astype(str).apply(len),
    "question": train_df["question"].astype(str).apply(len),
    "answer": train_df["answer"].astype(str).apply(len),
}

for key, series in length_stats.items():
    print(f"📏 {key} 평균 길이: {series.mean():.1f}")
    print(f"   최소/최대: {series.min()} / {series.max()}")

duplicates = train_df["question"].astype(str).duplicated().sum()
print(f"🧬 중복 question 개수: {duplicates}")


## 4. RAFT 템플릿 상태 확인
Distractor 문서가 잘 포함돼 있는지 확인합니다.

In [ ]:
import numpy as np

if "distractors" in train_df.columns:
    num_distractors = train_df["distractors"].apply(
        lambda x: len(x) if isinstance(x, (list, tuple, np.ndarray)) else 0
    )

    print(f"🎯 평균 distractor 수: {num_distractors.mean():.1f}")
    print(f"   distractor가 없는 샘플: {(num_distractors == 0).sum()}")
else:
    print("ℹ️ distractors 컬럼이 없습니다. RAFT 변환 없이 instruction-input-output 데이터만 존재합니다.")


## 5. 다음 단계
- 데이터가 정상이라면 main-practice/03에서 파인튜닝을 진행하세요.
- 텍스트 길이가 너무 길거나 distractor가 부족하면 실습 1(전처리)로 돌아가 규칙을 조정할 수 있습니다.